In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## Read multiple URLs

In [2]:
urls = ['https://www.pccomponentes.com/auriculares',
        'https://www.pccomponentes.com/teclados',
        'https://www.pccomponentes.com/ratones',
        'https://www.pccomponentes.com/sillas-gaming',
        'https://www.pccomponentes.com/monitores-pc',
        'https://www.pccomponentes.com/smartphone-moviles'
        ]
for url in urls:
        category = url.split('.com/')[-1]
        chrome = webdriver.Chrome()
        chrome.get(url)
        print("READING", url)
        try:
                element = WebDriverWait(chrome, 10).until(
                        EC.presence_of_element_located((By.ID, "cookiesAcceptAll"))
                )
                chrome.find_element(By.ID, 'cookiesAcceptAll').click()
        except:
                print("No hay cookies", url)
                pass
        
        grid = chrome.find_element(By.ID, 'product-grid')
        products = grid.find_elements(By.TAG_NAME, 'div')
        links = grid.find_elements(By.TAG_NAME, 'a')
        products_founded = 0
        print(f"LINKS FOUNDED: {len(links)}")
        for link in links:
                if link.get_attribute('data-product-id'):
                        product = link.find_element(By.CLASS_NAME, 'product-card')
                        name = ''
                        price = ''
                        reviews = ''
                        marketplace = False
                        if 'is-marketplace' in product.get_attribute('class'):
                                marketplace = True
                        try:
                                name = product.find_element(By.TAG_NAME, 'h3').text
                        except:
                                pass
                        try:
                                price = product.find_element(By.CLASS_NAME, 'product-card__price-container').text.replace('€','')
                        except:
                                pass
                        try:
                                reviews = product.find_element(By.CLASS_NAME, 'hKnlja').text.replace('(','').replace(')','')
                        except:
                                pass
                        
                        if name!='' and price!='' and reviews!='':
                                products_founded += 1
                                with open('products.csv', 'a', newline='') as file:
                                        file.write(f'{url};{category};{name};{price};{reviews};{marketplace}\n')
                        else:
                                print(f"Product error: ", link.get_attribute('href'))
                else:
                        print(f"Not product link: {link.get_attribute('href')}")   
        print(f"PRODUCTS FOUNDED: {products_founded}")
        chrome.quit()



READING https://www.pccomponentes.com/auriculares
LINKS FOUNDED: 40
https://www.pccomponentes.com/plantronics-cs540-monoaural
PRODUCTS FOUNDED: 0
READING https://www.pccomponentes.com/teclados
LINKS FOUNDED: 40
https://www.pccomponentes.com/logitech-mx-keys-mini-teclado-bluetooth-para-mac
PRODUCTS FOUNDED: 0
READING https://www.pccomponentes.com/ratones
LINKS FOUNDED: 43
https://www.pccomponentes.com/razer-pro-click-raton-ergonomico-inalambrico-16000-dpi-blanco
PRODUCTS FOUNDED: 43


: 

## Analyze

In [31]:
import pandas as pd

In [32]:
column_names = ['url', 'category', 'name', 'price', 'reviews', 'marketplace'] 
df = pd.read_csv('products.csv', sep=';',encoding='ISO-8859-1', names=column_names, decimal=',')
filtered = df.dropna(subset=['reviews'])

print(f"Products: {filtered['url'].count()}")
print()
print("REVIEWS")
print(f"AVG: {filtered['reviews'].mean()}")
print(f"MAX: {filtered['reviews'].max()}")
print(f"MIN: {filtered['reviews'].min()}")
print()
print("PRICE")
print(f"AVG: {filtered['price'].mean()}")
print(f"MAX: {filtered['price'].max()}")
print(f"MIN: {filtered['price'].min()}")

print()
top_10_least_reviews = filtered.nsmallest(10, 'reviews')
print("IMPROVE PRODUCTS")
print(top_10_least_reviews[['name','price','reviews']])

Products: 32

REVIEWS
AVG: 899.3125
MAX: 5490.0
MIN: 1.0

PRICE
AVG: 71.80312500000001
MAX: 179.0
MIN: 10.99

IMPROVE PRODUCTS
                                                 name   price  reviews
48  Mountain MacroPad Controlador para Contenido e...   94.99      1.0
44            Apple Magic Keyboard con Touch ID Plata  139.98     11.0
2   Logitech G Pro X TKL Lightspeed Teclado Inalám...  171.99     16.0
36  Apple Magic Keyboard con Touch ID y Teclado Nu...  179.00     17.0
40  Apple Magic Keyboard con Touch ID y Teclado Nu...  179.00     20.0
51   Logitech MX Keys Mini Teclado Bluetooth para Mac   81.99     22.0
5   Logitech MX Keys S Teclado Inalámbrico Bluetoo...   97.99     28.0
43  Logitech MX Keys S Teclado Inalámbrico Bluetoo...   97.99     28.0
12  Newskill Serike V2 Teclado Mecánico Gaming RGB...   97.99     74.0
46  Newskill Serike V2 TKL Teclado Mecánico Gaming...   82.99     98.0
